In [3]:
s = 1e-5

import pandas as pd
from sklearn import metrics
import tensorflow as tf
################################ MSE ################################
def MSE(y_true, y_pred):
    return tf.reduce_mean(tf.math.square(y_true-y_pred))

################################ BCE ################################
def BCE(y_true, y_pred):
    return -tf.reduce_mean(y_true*tf.math.log(y_pred+s)+(1-y_true)*tf.math.log(1-y_pred+s))

################################ WBCE ################################
def WBCE(y_true, y_pred):
    N = batch    # batch_size
    y1 = tf.reduce_sum(y_true)
    y0 = N-y1
    w1 = y0/N #N/y1
    w0 = y1/N #N/y0
    return -tf.reduce_mean(w1*y_true*tf.math.log(y_pred+s)+w0*(1-y_true)*tf.math.log(1-y_pred+s))

################################ TN/FP/FN/TP ################################
def confusion_matrix(y_true, y_pred):
    N = batch    # batch_size
    y1 = tf.reduce_sum(y_true)
    y0 = N-y1
    TN = N-tf.reduce_sum(y_true)-tf.reduce_sum(y_pred)+tf.reduce_sum(y_true*y_pred)
    FP = tf.reduce_sum(y_pred)-tf.reduce_sum(y_true*y_pred)
    FN = tf.reduce_sum(y_true)-tf.reduce_sum(y_true*y_pred)
    TP = tf.reduce_sum(y_true*y_pred)
    return N, y1, y0, TN, FP, FN, TP

################################ make_lists ################################
def make_lists():
    list_acc = []
    list_f1 = []
    list_gmean = []
    list_bacc = []
    list_pre = []
    list_rec = []
    list_spe = []
    return list_acc, list_f1, list_gmean, list_bacc, list_pre, list_rec, list_spe
    
############################### Results ###############################
def get_results(y_true, y_pred):
    TN = metrics.confusion_matrix(y_true, y_pred)[0,0]
    FP = metrics.confusion_matrix(y_true, y_pred)[0,1]
    FN = metrics.confusion_matrix(y_true, y_pred)[1,0]
    TP = metrics.confusion_matrix(y_true, y_pred)[1,1]
    acc = np.round((TP+TN)/(TP+TN+FP+FN),4)
    if TP+FP == 0:
        pre = 0
    else:
        pre = np.round(TP/(TP+FP),4)
    rec = np.round(TP/(TP+FN),4)
    spe = np.round(TN/(TN+FP),4)
    f1 = np.round(TP/(TP + 0.5*(FP+FN)),4)
    f05 = np.round(TP/(TP + 0.8*FP + 0.2*FN),4)
    f2 = np.round(TP/(TP + 0.2*FP + 0.8*FN),4)
    gmean = np.round(((TP/(TP+FN)) * (TN/(TN+FP)))**0.5,4)
    bacc = np.round(0.5*(TP/(TP+FN) + TN/(TN+FP)),4)
    
    list_acc.append(acc)
    list_f1.append(f1)
    list_gmean.append(gmean)
    list_bacc.append(bacc)
    list_pre.append(pre)
    list_rec.append(rec)
    list_spe.append(spe)

################################ SPL ################################
def splitter(y_pred):
    return (0.5)**2-(y_pred-0.5)**2

# =================================== Fbeta =================================== #
################################ Pure_Fbeta ################################
def Pure_Fbeta(y_true, y_pred):
    b = 1 
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    F_beta = ((1+b**2)*TP) / ((b**2)*y1 + tf.reduce_sum(y_pred)+s)  # (1+b**2)TP/((1+b**2)TP+FP+b**2FN)
    return 1-F_beta

################################ Any_Fbeta ################################
def Any_Fbeta(y_true, y_pred):
    b = 1 
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    F_beta = ((1+b**2)*TP) / ((b**2)*y1 + tf.reduce_sum(y_pred)+s)  # (1+b**2)TP/((1+b**2)TP+FP+b**2FN)
    return 1-F_beta

################################ WBCEFL ################################
def WBCEFL(y_true, y_pred):
    b = 1
    WBCEloss = WBCE(y_true, y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    F_beta = ((1+b**2)*TP) / ((b**2)*y1 + tf.reduce_sum(y_pred)+s)  # (1+b**2)TP/((1+b**2)TP+FP+b**2FN)
    return (1-r)*WBCEloss+(r)*(1-F_beta)

################################ SPLFL ################################
def SPLFL(y_true, y_pred):
    b = 1
    SPL = splitter(y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    F_beta = ((1+b**2)*TP) / ((b**2)*y1 + tf.reduce_sum(y_pred)+s)  # (1+b**2)TP/((1+b**2)TP+FP+b**2FN)
    return (1-w)*SPL+(w)*(1-F_beta)

# =================================== Gmean =================================== #
################################ Pure_Gmean ################################
def Pure_Gmean(y_true, y_pred):
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    sur_gmean = (TP*TN)/(y1*y0+s)
    return 1-sur_gmean

################################ Any_Gmean ################################
def Any_Gmean(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    sur_gmean = (TP*TN)/(y1*y0+s)
    return 1-sur_gmean

################################ WBCEGL ################################
def WBCEGL(y_true, y_pred):
    WBCEloss = WBCE(y_true, y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    sur_gmean = (TP*TN)/(y1*y0+s)
    return (1-r)*WBCEloss+(r)*(1-sur_gmean)

################################ SPLGL ################################
def SPLGL(y_true, y_pred):
    SPL = splitter(y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    sur_gmean = (TP*TN)/(y1*y0+s)
    return (1-w)*SPL+(w)*(1-sur_gmean)

# =================================== BAccu =================================== #
################################ Pure_BAccu ################################
def Pure_BAccu(y_true, y_pred):
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    baccu = (y0*TP+y1*TN) / (2*y1*y0+s)
    return 1-baccu

################################ Any_BAccu ################################
def Any_BAccu(y_true, y_pred):
    y_pred = 1/(1+tf.math.exp(-L*(y_pred-0.5)))
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    baccu = (y0*TP+y1*TN) / (2*y1*y0+s)
    return 1-baccu

################################ WBCEBL ################################
def WBCEBL(y_true, y_pred):
    WBCEloss = WBCE(y_true, y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    baccu = (y0*TP+y1*TN) / (2*y1*y0+s)
    return (1-r)*WBCEloss+(r)*(1-baccu)

################################ SPLBL ################################
def SPLBL(y_true, y_pred):
    SPL = splitter(y_pred)
    N, y1, y0, TN, FP, FN, TP = confusion_matrix(y_true, y_pred)
    baccu = (y0*TP+y1*TN) / (2*y1*y0+s)
    return (1-w)*SPL+(w)*(1-baccu)

In [ ]:
res = pd.DataFrame({'MSE':[0, 0, 0, 0, 0, 0, 0]}, index = ['Acc','F1','G_Mean','B_Acc','Pre','Rec','Spe'])
res

# Step1: Select One Dataset

## 1)SPAM

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#  https://www.kaggle.com/uciml/sms-spam-collection-dataset
data = pd.read_csv('spam.csv', encoding='latin1')
print('sample number:',len(data))

In [ ]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data[:5]

In [ ]:
# missing?
data.isnull().values.any()

In [ ]:
# unique?
data['v2'].nunique()

In [ ]:
# delete duplicate
data.drop_duplicates(subset=['v2'], inplace=True)
data.info()

In [ ]:
# delete duplicate
data.drop_duplicates(subset=['v2'], inplace=True)
data.info()

In [ ]:
data.drop(data[:700][data['v1'][:700] == 1].index, inplace=True)
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data

In [ ]:
data['v1'].value_counts()

In [ ]:
X = data['v2']
y = data['v1']
y = y.astype(float)

In [ ]:
# integer encoding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_encoded = tokenizer.texts_to_sequences(X)

In [ ]:
# index (more frequent, smaller number given)
word_to_index = tokenizer.word_index
# print(len(word_to_index), word_to_index)

In [ ]:
# word group for padding
vocab_size = len(word_to_index) + 1
print(vocab_size)

In [ ]:
# word group for padding
vocab_size = len(word_to_index) + 1
print(vocab_size)

In [ ]:
# padding
max_len = maxlen
X_padded = pad_sequences(X_encoded, maxlen = max_len)
X_padded.shape

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

epochs = 20
batch = 64

## 2)Reuters News

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import reuters

In [ ]:
vocab_size = 1000
maxlen = 100
(XX, yy), (X_dummy, y_dummy) = reuters.load_data(num_words=vocab_size, test_split=0)

print(len(XX))
print(len(X_dummy))
num_classes = len(set(yy))
print(num_classes)

In [ ]:
word_to_index = reuters.get_word_index()
# print(len(word_to_index), word_to_index)

In [ ]:
# padding
max_len = maxlen
X_padded = pad_sequences(XX, maxlen = max_len)
X_padded.shape

In [ ]:
# Picking only label 3->'0' & 1->'1'
idx_1 = []
idx_3 = []
for i in range(len(yy)):
    if list(yy)[i] == 1:
        idx_1.append(i)
    if list(yy)[i] == 3:
        idx_3.append(i)
print(len(idx_1), len(idx_3))

In [ ]:
idx = idx_1[:100] + idx_3[:800]
idx.sort()

In [ ]:
X = []
y = []
for i in idx:
    X.append(X_padded[i])
    y.append(yy[i])
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

In [ ]:
y3 = (y == 1)
y3 = list(y3)

y_3 = []
for i in range(len(y3)):
    if y3[i] == True:
        y_3.append(1)
    else:
        y_3.append(0)
y_3 = np.array(y_3)

print(pd.Series(y_3).value_counts())

In [ ]:
X_padded = X
y_3 = y_3.astype(float)
print(X_padded.shape, y_3.shape)

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

epochs = 20
batch = 128

## 3)IMDB

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

In [ ]:
vocab_size = 1000
max_len = 100

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

In [ ]:
# Making lists of index
idx_0 = []
idx_1 = []
for i in range(len(y_train)):
    if list(y_train)[i] == 0:
        idx_0.append(i)
    if list(y_train)[i] == 1:
        idx_1.append(i)
print(len(idx_0), len(idx_1))

In [ ]:
idx = idx_0[:450] + idx_1[:150]
idx.sort()
# print(len(idx), idx)

In [ ]:
X = []
y = []
for i in idx:
    X.append(X_train[i])
    y.append(y_train[i])
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

In [ ]:
word_to_index = imdb.get_word_index()
# print(len(word_to_index), word_to_index)

In [ ]:
# padding
X_padded = pad_sequences(X, maxlen = max_len)
X_padded.shape

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

epochs = 20
batch = 64

# Step2: Select One Architecture

## 1)RNN(LSTM)

In [31]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM, GRU
from tensorflow.keras.models import Sequential
from keras import optimizers

embedding_dim = 32
hidden_units = 32

def create_model():
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))
    # model.add(SimpleRNN(hidden_units))
    model.add(LSTM(hidden_units))
    # model.add(GRU(hidden_units))
    model.add(Dense(1, activation='sigmoid'))    
    return model

In [1]:
mse_acc = []
mse_f1 = []
mse_gmean = []
mse_bacc = []
mse_pre = []
mse_rec = []
mse_spe = []

for i in range(5):
    print('#'*50,'{0}th repeat'.format(i+1),'#'*50)
    list_acc, list_f1, list_gmean, list_bacc, list_pre, list_rec, list_spe = make_lists()

    n_iter=0
    ###################### MLP (sigmoid // MSE) ##############################
    for train_index, test_index in skf.split(X_padded, y):
        n_iter += 1
        X_train = X_padded[train_index]
        y_train= y[train_index]
        X_test = X_padded[test_index]
        y_test= y[test_index]
#         print('#'*50,'{0}th CV'.format(n_iter),'#'*50)
    #     X_train = np.array(X_train)
    #     y_train = np.array(y_train)
    #     y_train = y_train.astype(float)
    #     X_test = np.array(X_test)
    #     y_test = np.array(y_test)
    #     y_test = y_test.astype(float)

        model = create_model()
        early_stopping = EarlyStopping(monitor='loss', patience=0, min_delta=1e-4)
    #     check_point = ModelCheckpoint('best_model.h5', monitor="val_loss", save_best_only=True, save_weights_only=True)
        opt = optimizers.Adam(learning_rate = 0.001)
        model.compile(optimizer=opt, loss=MSE, metrics=['accuracy'])
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), verbose=0, epochs=epochs, batch_size = batch, ) 
                            #callbacks=[early_stopping]) #, check_point])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()
        predicted = []
    #     model.load_weights('best_model.h5')  # Best Model by Check_Point
        result = model.predict(X_test)
        for i in range(X_test.shape[0]):
            if result[i] <= 0.5:
                predicted.append(0)
            else:
                predicted.append(1)
        get_results(y_test, predicted)
    print("Acc:{}\nF1:{}\nGM:{}\nBA:{}\nPRE:{}\nREC:{}\nSPE:{}\n".format(np.mean(list_acc),np.mean(list_f1),np.mean(list_gmean),
                                                                         np.mean(list_bacc),np.mean(list_pre),np.mean(list_rec),
                                                                         np.mean(list_spe)))     
    mse_acc.append(np.mean(list_acc))
    mse_f1.append(np.mean(list_f1))
    mse_gmean.append(np.mean(list_gmean))
    mse_bacc.append(np.mean(list_bacc))
    mse_pre.append(np.mean(list_pre))
    mse_rec.append(np.mean(list_rec))
    mse_spe.append(np.mean(list_spe))
               
res['MSE'] = [np.mean(mse_acc), np.mean(mse_f1), np.mean(mse_gmean), np.mean(mse_bacc), 
              np.mean(mse_pre), np.mean(mse_rec), np.mean(mse_spe)]
res  

In [2]:
print("AC:", np.round(np.mean(mse_acc),4),'±',np.round(np.std(mse_acc),4))
print("F1:", np.round(np.mean(mse_f1),4),'±',np.round(np.std(mse_f1),4))
print("GM:", np.round(np.mean(mse_gmean),4),'±',np.round(np.std(mse_gmean),4))
print("BA:", np.round(np.mean(mse_bacc),4),'±',np.round(np.std(mse_bacc),4))
print("PRE:", np.round(np.mean(mse_pre),4),'±',np.round(np.std(mse_pre),4))
print("REC:", np.round(np.mean(mse_rec),4),'±',np.round(np.std(mse_rec),4))
print("SPE:", np.round(np.mean(mse_spe),4),'±',np.round(np.std(mse_spe),4))

In [ ]:
# For each loss, do above
# LOSS = [MSE, BCE, WBCE, Pure_Fbeta, Any_Fbeta, WBCEFL, SPLFL, 
#         Pure_Gmean, Any_Gmean, WBCEGL, SPLGL, 
#         Pure_BAccu, Any_BAccu, WBCEBL, SPLBL ]

In [1]:
# Final Results
res

## 2)CNN

In [ ]:
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Embedding, Dropout, MaxPooling1D
from tensorflow.keras.models import Sequential
from keras import optimizers

embedding_dim = 32
dropout_ratio = 0.3
num_filters = 32
kernel_size = 5

def create_model():
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))
    model.add(Dropout(dropout_ratio))
    model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(dropout_ratio))
    model.add(Dense(1, activation='sigmoid'))    
    return model

In [ ]:
mse_acc = []
mse_f1 = []
mse_gmean = []
mse_bacc = []
mse_pre = []
mse_rec = []
mse_spe = []

for i in range(5):
    print('#'*50,'{0}th repeat'.format(i+1),'#'*50)
    list_acc, list_f1, list_gmean, list_bacc, list_pre, list_rec, list_spe = make_lists()

    n_iter=0
    ###################### MLP (sigmoid // MSE) ##############################
    for train_index, test_index in skf.split(X_padded, y):
        n_iter += 1
        X_train = X_padded[train_index]
        y_train= y[train_index]
        X_test = X_padded[test_index]
        y_test= y[test_index]
#         print('#'*50,'{0}th CV'.format(n_iter),'#'*50)
    #     X_train = np.array(X_train)
    #     y_train = np.array(y_train)
    #     y_train = y_train.astype(float)
    #     X_test = np.array(X_test)
    #     y_test = np.array(y_test)
    #     y_test = y_test.astype(float)

        model = create_model()
        early_stopping = EarlyStopping(monitor='loss', patience=0, min_delta=1e-4)
    #     check_point = ModelCheckpoint('best_model.h5', monitor="val_loss", save_best_only=True, save_weights_only=True)
        opt = optimizers.Adam(learning_rate = 0.005)
        model.compile(optimizer=opt, loss=MSE, metrics=['accuracy'])
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), verbose=0, epochs=epochs, batch_size = batch, )
                            #callbacks=[early_stopping]) #, check_point])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()
        predicted = []
    #     model.load_weights('best_model.h5')  # Best Model by Check_Point
        result = model.predict(X_test)
        for i in range(X_test.shape[0]):
            if result[i] <= 0.5:
                predicted.append(0)
            else:
                predicted.append(1)
        get_results(y_test, predicted)
    print("Acc:{}\nF1:{}\nGM:{}\nBA:{}\nPRE:{}\nREC:{}\nSPE:{}\n".format(np.mean(list_acc),np.mean(list_f1),np.mean(list_gmean),
                                                                         np.mean(list_bacc),np.mean(list_pre),np.mean(list_rec),
                                                                         np.mean(list_spe)))     
    mse_acc.append(np.mean(list_acc))
    mse_f1.append(np.mean(list_f1))
    mse_gmean.append(np.mean(list_gmean))
    mse_bacc.append(np.mean(list_bacc))
    mse_pre.append(np.mean(list_pre))
    mse_rec.append(np.mean(list_rec))
    mse_spe.append(np.mean(list_spe))
               
res['MSE'] = [np.mean(mse_acc), np.mean(mse_f1), np.mean(mse_gmean), np.mean(mse_bacc), 
              np.mean(mse_pre), np.mean(mse_rec), np.mean(mse_spe)]
res 

In [ ]:
print("AC:", np.round(np.mean(mse_acc),4),'±',np.round(np.std(mse_acc),4))
print("F1:", np.round(np.mean(mse_f1),4),'±',np.round(np.std(mse_f1),4))
print("GM:", np.round(np.mean(mse_gmean),4),'±',np.round(np.std(mse_gmean),4))
print("BA:", np.round(np.mean(mse_bacc),4),'±',np.round(np.std(mse_bacc),4))
print("PRE:", np.round(np.mean(mse_pre),4),'±',np.round(np.std(mse_pre),4))
print("REC:", np.round(np.mean(mse_rec),4),'±',np.round(np.std(mse_rec),4))
print("SPE:", np.round(np.mean(mse_spe),4),'±',np.round(np.std(mse_spe),4))

In [ ]:
# For each loss, do above
# LOSS = [MSE, BCE, WBCE, Pure_Fbeta, Any_Fbeta, WBCEFL, SPLFL, 
#         Pure_Gmean, Any_Gmean, WBCEGL, SPLGL, 
#         Pure_BAccu, Any_BAccu, WBCEBL, SPLBL ]

In [ ]:
# Final Results
res